In [56]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import itertools
from datetime import datetime, timedelta

data_1 = '{"default_view":"this_month","impacts":[3,2,1,0],"event_types":[2,4,10],"currencies":[1,2,3,4,5,6,7,8,9]}'
data_2 = '{"default_view":"next_month","impacts":[3,2,1,0],"event_types":[2,4,10],"currencies":[1,2,3,4,5,6,7,8,9]}'

In [51]:
# 目标页面 URL
url = f'https://www.forexfactory.com/calendar?month=this'

# 设置请求头，模拟浏览器访问，防止被识别为爬虫
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}


In [57]:
response = requests.get(url, headers=headers, data=data_2)
rows = None
# 检查响应状态码
response.status_code

200

In [58]:
def parse_economic_date(date_string):
    current_year = datetime.now().year
    return datetime.strptime(f"{date_string} {current_year}", "%a%b %d %Y").strftime("%Y-%m-%d")


In [59]:
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    events_table = soup.find('table', {'class': 'calendar__table'})

    rows = events_table.find_all('tr')

table_results = {}

for index, row in enumerate(rows):
    columns = row.find_all('td')
    row_result = {}
    for idx, col in enumerate(columns):
        try:
            col_name = col.get('class')[1].replace('calendar__', '')
            if col_name == 'impact':
                col_value = col.find('span').get('class')[1]
            else:
                col_value = col.get_text(strip=True)
            row_result[col_name] = col_value
        except Exception as e:
            pass
    table_results[index] = row_result

columns = [value.keys() for key, value in table_results.items()]
columns = list(set(list(itertools.chain(*columns))))
columns.append('Index')

df = pd.DataFrame({key: [] for key in columns})

for index, values in table_results.items():
    curr_row = {}
    for col in columns:
        if col == 'Index':
            curr_row[col] = [int(index)]
        else:
            curr_row[col] = [values.get(col, None)]
    new_row = pd.DataFrame(curr_row)  # Create a DataFrame for the new row

    df = pd.concat([df, new_row], ignore_index=True)

df = df.sort_values(by=['Index']).reset_index(drop=True)
df = df[df['event'].notna()]
df['date'] = df['date'].ffill()
df['date'] = df['date'].apply(parse_economic_date)

In [60]:
df

,date,sub,currency,detail,event,graph,actual,previous,time,forecast,impact,Index
4,2025-03-01,,USD,,President Trump Speaks,,,,1:15am,,icon--ff-impact-red,4.0
5,2025-03-01,,CNY,,Manufacturing PMI,,50.2,49.1,9:30am,49.9,icon--ff-impact-red,5.0
6,2025-03-01,,CNY,,Non-Manufacturing PMI,,50.4,50.2,,50.4,icon--ff-impact-ora,6.0
7,2025-03-01,,USD,,FOMC Member Goolsbee Speaks,,,,11:15am,,icon--ff-impact-yel,7.0
11,2025-03-03,,NZD,,Overseas Trade Index q/q,,3.1%,2.5%,5:45am,1.5%,icon--ff-impact-yel,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...
420,2025-03-31,,GBP,,M4 Money Supply m/m,,,,4:30pm,,icon--ff-impact-yel,420.0
421,2025-03-31,,GBP,,Mortgage Approvals,,,,,,icon--ff-impact-yel,421.0
422,2025-03-31,,GBP,,Net Lending to Individuals m/m,,,,,,icon--ff-impact-yel,422.0
423,2025-03-31,,EUR,,Italian Prelim CPI m/m,,,,5:00pm,,icon--ff-impact-yel,423.0
